In [1]:
import pandas as pd
from tqdm import tqdm
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from scipy.sparse import csr_matrix, hstack
from sklearn.svm import SVC

## N-Gram construction

In [2]:
data_train = pd.read_csv('../input/task1a3/sentiment_train.csv')
data_test = pd.read_csv('../input/task1a3/sentiment_test.csv')

In [3]:
df_TrainFeature_without_ngram=pd.read_csv("../input/nlp-a3-ques1-sklearn-features/TrainFeatures_without_ngrams")
df_TestFeature_without_ngram=pd.read_csv("../input/nlp-a3-ques1-sklearn-features/TestFeatures_without_ngrams")

In [4]:
tweets_train=data_train['5']
tweets_test=data_test['5']

tweets_train=tweets_train.apply(lambda s:s.lower())
tweets_test=tweets_test.apply(lambda s:s.lower())

TrainRange=50000
TestRange=len(tweets_test)

tweets_train=tweets_train[0:TrainRange]
tweets_test=tweets_test[0:TestRange]

In [5]:
Alltweets=tweets_train
Alltweets=Alltweets.append(tweets_test)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
# ngram_range parameter (1,2) means that unigram and bigram will be taken
# Count Vecotrizer automatically preprocess the tweets
vectorizer = CountVectorizer(ngram_range=(1,4),min_df=10)

In [7]:
# fit was used to learn the vocabulary
TempVector=vectorizer.fit(Alltweets)

In [9]:
# Getting the vocabulary for ngrams
Vocab_ngrams=TempVector.get_feature_names()
len(Vocab_ngrams)

11250

In [10]:
Vocab={}
for i in range(len(Vocab_ngrams)):
    Vocab[i]=Vocab_ngrams[i]

In [11]:
import pickle
file=open("Dictionay_ngram_features_50k",'wb')
pickle.dump(Vocab,file)
file.close()

In [12]:
# Now we can use this vocabulary to individually create ngrams for train and test set
# fit transform is used here as we want the counts too
vectorizer2 = CountVectorizer(ngram_range=(1,4), vocabulary=Vocab_ngrams)
Train_ngrams=vectorizer2.fit_transform(tweets_train)
Test_ngrams=vectorizer2.fit_transform(tweets_test)

In [13]:
import pickle
file=open("Train_ngrams_50k",'wb')
pickle.dump(Train_ngrams,file)
file.close()
file=open("Test_ngrams_50ktrain",'wb')
pickle.dump(Test_ngrams,file)
file.close()

In [14]:
csr_TrainFeature_without_ngram=csr_matrix(df_TrainFeature_without_ngram[0:TrainRange])
csr_TestFeature_without_ngram=csr_matrix(df_TestFeature_without_ngram[0:TestRange])
TrainFeatureFinal=hstack([Train_ngrams,csr_TrainFeature_without_ngram])
TestFeatureFinal=hstack([Test_ngrams,csr_TestFeature_without_ngram])

In [15]:
DTmodel=DecisionTreeClassifier()

In [16]:
DTmodel.fit(TrainFeatureFinal,data_train['0'][0:TrainRange])

DecisionTreeClassifier()

In [17]:
import pickle
file=open("DTmodel_50k",'wb')
pickle.dump(DTmodel,file)
file.close()

In [19]:
y2=DTmodel.predict(TestFeatureFinal)
y2.shape

(480000,)

In [20]:
import pickle
file=open("y_DT_50k",'wb')
pickle.dump(y2,file)
file.close()

In [25]:
clf = SVC(verbose=True)
clf.fit(TrainFeatureFinal,data_train['0'][0:TrainRange])

[LibSVM]

SVC(verbose=True)

In [26]:
import pickle
file=open("SVCmodel50ktrain",'wb')
pickle.dump(clf,file)
file.close()

In [27]:
ySCV=clf.predict(TestFeatureFinal)

In [29]:
import pickle
file1=open("ySCV50ktrain",'wb')
pickle.dump(ySCV,file1)
file1.close()

In [21]:
from sklearn.neural_network import MLPClassifier

clf2 = MLPClassifier(max_iter=300,verbose=True)
clf2.fit(TrainFeatureFinal,data_train['0'][0:TrainRange])

Iteration 1, loss = 0.47223788
Iteration 2, loss = 0.39944030
Iteration 3, loss = 0.35827391
Iteration 4, loss = 0.32101355
Iteration 5, loss = 0.28645006
Iteration 6, loss = 0.25407663
Iteration 7, loss = 0.21971458
Iteration 8, loss = 0.18896948
Iteration 9, loss = 0.15951808
Iteration 10, loss = 0.13196118
Iteration 11, loss = 0.11430425
Iteration 12, loss = 0.09454833
Iteration 13, loss = 0.07680971
Iteration 14, loss = 0.06594767
Iteration 15, loss = 0.05872366
Iteration 16, loss = 0.04925504
Iteration 17, loss = 0.04767966
Iteration 18, loss = 0.03809580
Iteration 19, loss = 0.03311983
Iteration 20, loss = 0.03060653
Iteration 21, loss = 0.02958855
Iteration 22, loss = 0.02779880
Iteration 23, loss = 0.02760848
Iteration 24, loss = 0.02220098
Iteration 25, loss = 0.02152374
Iteration 26, loss = 0.02021733
Iteration 27, loss = 0.02132274
Iteration 28, loss = 0.01823085
Iteration 29, loss = 0.01740286
Iteration 30, loss = 0.01525037
Iteration 31, loss = 0.01518245
Iteration 32, los

MLPClassifier(max_iter=300, verbose=True)

In [22]:
import pickle
file2=open("MLPmodel_50ktrain",'wb')
pickle.dump(clf2,file2)
file2.close()

In [1]:
y4=clf2.predict(TestFeatureFinal)

In [24]:
import pickle
file3=open("yMLP50ktrain",'wb')
pickle.dump(y4,file3)
file3.close()

In [ ]:
import pickle
file3=open("TrainFeatureFinal",'wb')
pickle.dump(TrainFeatureFinal,file3)
file3.close()
file3=open("TestFeatureFinal",'wb')
pickle.dump(TestFeatureFinal,file3)
file3.close()